In [ ]:
!pip install pycuda

In [ ]:
import numpy as np
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

## Multiplication matricielle 

$$
\begin{bmatrix}
 a_{11} &  \dots  & a_{1n} \\
 \vdots &  \ddots & \vdots \\
 a_{m1} &  \dots  & a_{mn}
\end{bmatrix}
\times
\begin{bmatrix}
 b_{11} & \dots  & b_{1p} \\
 \vdots & \ddots & \vdots \\
 b_{n1} & \dots  & b_{np}
\end{bmatrix}
=\begin{bmatrix}
  c_{11} &  \dots  & c_{1m} \\
 \vdots &  \ddots & \vdots \\
 a_{n1} &  \dots  & c_{mp}
\end{bmatrix}
$$

$$
c_{ij}=\sum_{k=1}^na_{ik}b_{kj}
$$

## Fonction mult_mat en python

In [ ]:
def mult_mat(A,B):
  adim1,adim2 = A.shape
  bdim1,bdim2 = B.shape
  C=np.zeros((adim1,bdim2))
  #adim2 == bdim1
  for i in range(adim1):
    for j in range(bdim2):
      for k in range(adim2):
        C[i,j] += A[i,k] * B[k,j]
  return C

## Compléter le kernel

In [ ]:
kernels = SourceModule("""
__global__ void mult_mat(float *A,float *B,float *C, int width)
{
   //indice de la ieme ligne de A
   int Ai = TODO ;
   //indice de la jeme colonne de B
   int Bj = TODO ;
   //indice de la position C[i,j] 
   int Cij = TODO ;
   C[Cij] = 0.0f;
   // parcours de la ligne et de la colonne
   for (int j=0;j<width;j++)
      TODO ;
}
""")

## Initialisation des matrices sur le host

In [ ]:
h_a=np.random.randn(32,32)
h_b=np.random.randn(32,32)
h_a=h_a.astype(np.float32)
h_b=h_b.astype(np.float32)
h_c=...

## Lecture du kernel

In [ ]:
kmult_mat=kernels.get_function("mult_mat")

## Compléter les allocations mémoire et l'écriture sur le device

In [ ]:
g_a=cuda.mem_alloc(h_a.nbytes)
g_b=cuda.mem_alloc(h_b.nbytes)
g_c=cuda.mem_alloc(...)
cuda.memcpy_htod(g_a,h_a)
cuda.memcpy_htod(g_b,h_b)

## Appeler le kernel avec le bon block

In [ ]:
width=np.int32(...)
kmult_mat(g_a,g_b,g_c,width,block=(...,...,...))

## Récupération du résultat sur le host

In [ ]:
cuda.memcpy_dtoh(h_c,g_c)

## Libération de la mémoire sur le devcice

In [ ]:
g_a.free()
g_b.free()
g_c.free()

## Affichage résultat

In [ ]:
print(np.dot(h_a,h_b))
print(h_c)